# __Calcolo delle distribuzioni stazionarie__
Osserviamo come viene calcolato il vettore $\pi$ della distrubuzione stazionaria in varie librerie e <br>
implementiamo vari metodi e confrontiamo tempi e precisione.

In [ ]:
import generazione_catene, utility, metodi_stazionaria, complessità

import matplotlib.pyplot as plt

from quantecon import gth_solve
from pydtmc import MarkovChain

## Funzioni

### `Generazione di P`

#### P irriducibile

> **Nota sulla generazione**: questa funzione è adatta per costruire matrici di transizione $P$ casuali, con un grado controllabile di sparsità, garantendo l'irriducibilità tramite controllo di forte connessione su $G(P)$.  
>
> Tuttavia, quando il numero di zeri richiesto si avvicina al massimo teorico $n(n-1)$, trovare una matrice fortemente connessa diventa sempre più difficile, e i tempi di generazione crescono drasticamente.  
> Inoltre, anche se $P$ risulta irriducibile, un numero troppo basso di archi può portare a **periodicità**, rendendo il **Power Method inefficace**.
>
> In questi casi estremi è preferibile usare una generazione alternativa con struttura più controllata (es. `genera_P_k_random`), che garantisce a priori la forte connessione e, con un minimo numero di archi extra, anche l’aperiodicità.

In [ ]:
P = generazione_catene.genera_P_irriducibile(5,5)
print(P)

#### P irriducibile con k archi per nodo

> **Nota sulla generazione**: questo approccio garantisce la $\textbf{forte connessione}$ della matrice $P$ per costruzione,  
> grazie all’arco obbligato $( i \rightarrow (i+1) \mod n )$ che forma un ciclo completo tra tutti i nodi.  
>
> Aggiungendo poi \( k-1 \) archi casuali per riga, si rompe facilmente anche la $\textbf{periodicità}$ del ciclo, rendendo la matrice adatta all'uso del Power Method.  
>
> Questo metodo è particolarmente utile quando si desidera generare matrici $\emph{molto sparse}$, ma con garanzie strutturali.  
> A differenza della funzione `genera_P_irriducibile`, qui non è necessario alcun controllo esplicito sulla connettività,  
> e i tempi di generazione restano contenuti anche con pochissimi archi (es. $ k = 2 $).

In [ ]:
P = generazione_catene.genera_P_k_random(5,2)
print(P)

### `Generazione di Q`

#### Q irriducibile

> **Nota sulla generazione**: questa funzione genera matrici generatore $Q$ **irriducibili** (fortemente connesse)  
> a partire da una struttura sparsa con zeri distribuiti casualmente fuori diagonale.  
>
> L’algoritmo impone un massimo di $n(n - 2)$ zeri fuori diagonale, per garantire che sia matematicamente possibile ottenere  
> un grafo fortemente connesso.  
>
> **Osservazioni:**
> - Quando il numero di zeri si avvicina al massimo consentito, la probabilità di ottenere una $Q$ fortemente connessa si riduce drasticamente, e può essere necessario un numero elevato di tentativi.
> - Anche se la matrice è fortemente connessa, un numero troppo basso di archi rende probabile la **periodicità** della catena. Tuttavia, nei test questa si è rivelata molto rara.
> - Questa funzione è più robusta rispetto alla generazione di $P$ sparse: riesce a garantire connettività anche in presenza di elevata sparsità.
>
> Utile quando si desiderano CTMC reversibili con controllo sulla sparsità e mantenimento dell’ergodicità.

In [ ]:
Q = generazione_catene.genera_Q_irriducibile(5,5)
print(Q)

#### Q irriducibile con k archi per nodo

> **Nota sulla generazione**: questa funzione costruisce matrici generatore $Q$ irriducibili,  
> fissando per ogni riga un arco obbligato $i \to (i+1) \bmod n$ per garantire la connessione,  
> e aggiungendo $k-1$ archi off-diagonali scelti a caso. I tassi sono estratti da $(0, \texttt{tasso\_massimo}]$,  
> mentre la diagonale viene calcolata per ottenere righe a somma nulla, come richiesto per i generatori CTMC. <br>
> La struttura ciclica garantisce la connettività, ma **non assicura di per sé l’aperiodicità**:  
> tuttavia, questa non è un requisito per metodi come GTH, sistemi lineari o autovalori.  
> Quando si usa il Power Method, invece, la matrice $Q$ viene trasformata con l’uniformizzazione  
> $P = I + Q / B$, che introduce autoloop e rompe la periodicità, rendendo il metodo sempre applicabile.

In [ ]:
Q = generazione_catene.genera_Q_k_random(5,2)
print(Q)

### `Bound per le k-iterazioni`

Stima precisa del bound superiore sul numero di iterazioni $k$ affinché
$$
\|\pi^{(k)} - \pi\|_1 < \text{tol}
$$
usando il bound spettrale completo derivato dalla decomposizione spettrale
di $P^\top$ (o della matrice uniformizzata, nel caso continuo).

Il bound teorico utilizzato è:
$$
\|\pi^{(k)} - \pi\|_1 \le \sqrt{n} \cdot \left(\sum_{i=2}^{n} \left|\frac{\alpha_i}{\alpha_1}\right|^2 \cdot \left|\frac{\lambda_i}{\lambda_1}\right|^{2k}\right)^{1/2}
$$
dove:
- $\lambda_i$ sono gli autovalori (in ordine di modulo decrescente),
- $\alpha_i$ sono i coefficienti nella decomposizione $\pi^{(0)} = \sum \alpha_i v_i$ degli autovettori destri di $P^\top$,
- $\pi^{(0)}$ è il vettore iniziale uniforme,
- $n$ è la dimensione della matrice.

Da cui segue che, per ottenere un errore inferiore a una data tolleranza, si cerca il minimo intero $k$ tale che:
$[
k \ge \min \left\{ k \in \mathbb{N} \;\middle|\; \sqrt{n} \cdot \left( \sum_{i=2}^{n} \left| \frac{\alpha_i}{\alpha_1} \right|^2 \cdot \left| \frac{\lambda_i}{\lambda_1} \right|^{2k} \right)^{1/2} < \text{tol} \right\}
]$

Questa stima non può essere risolta in forma chiusa, per questo motivo, nel nostro codice cerchiamo iterativamente <br>
il minimo intero $k$ tale che il bound sia minore della tolleranza desiderata.

In [ ]:
P = generazione_catene.genera_P_irriducibile(10,80)
k = metodi_stazionaria.solve_via_power_numpy(P)
k_bound = utility.stima_k_necessario_precisa(P)

print(k)
print(k_bound)

## Quantecon

### Quantecon usa `gth_solve()` che implementa l'algoritmo GTH.

`gth_solve()` accetta matrici `P` o `Q`, cioè che siano di tipo **Metzler**,  
ovvero con **elementi off-diagonali non negativi**.

Effettua una **copia** della matrice di input in `A`,  
che viene modificata **in-place** per calcolare una **fattorizzazione numericamente stabile**,  
**senza uso di pivot**.

Ad ogni passo `k`:
- calcola la **scala** come somma degli elementi a destra del pivot nella riga `k`
- **normalizza la colonna `k` sotto la diagonale** dividendo per questa scala
- **manipola la sottomatrice in basso a destra** aggiornando ogni elemento secondo:
$A[i,j] += A[i,k] * A[k,j]$   per $i > k, j > k$ <br>

---

Per calcolare il vettore stazionario $\pi$,  
la backward substitution parte da `π[n-1] = 1` e risale:
$π[k] = ∑ A[i,k] * π[i]$   per $i = k+1 … n-1$ <br>

Ovvero, per ogni `π[k]`, si usano **gli elementi nella colonna `k` sotto la diagonale**,  
e per ciascuno di essi (posizione `A[i, k]`), si **moltiplica per il valore `π[i]` già calcolato**,  
dove `i` è proprio l’indice di riga di quell’elemento.

Si sommano tutti questi prodotti per ottenere `π[k]`.

Alla fine, `π` viene normalizzato: `π /= π.sum()`.

### Confronto su matrice stocastica P con gth_solve usata in `Quantecon` e la nostra implementazione

In [ ]:
P = generazione_catene.genera_P_k_random(5, 2)

pi_builtin = gth_solve(P)
pi_manual = metodi_stazionaria.my_gth_solve(P)

print("π (quantecon):", pi_builtin, "\n")
print("π (nostra GTH):", pi_manual, "\n")
print("Verifica π • P (nostra GTH):", pi_manual @ P)

### Confronto su matrice generatore Q

In [ ]:
Q = generazione_catene.genera_Q_irriducibile(5, 4)

pi_builtin = gth_solve(Q)
pi_q = metodi_stazionaria.my_gth_solve(Q)

print("π (quantecon):", pi_builtin, "\n")
print("π per Q (nostra GTH):", pi_q, "\n")
print("Verifica π @ Q:", pi_q @ Q)  # Deve essere vicino a 0

## PyDTMC

### PyDTMC: calcolo di π e test di reversibilità

* **Calcolo di π**  
  PyDTMC impiega lo stesso algoritmo **GTH** di *quantecon*, ma lo applica soltanto a
  matrici di transizione **P** (catene DTMC), dopo aver verificato che la somma di
  ogni riga sia pari a 1.

* **Reversibilità**  
  Una volta ottenuta la distribuzione stazionaria $\pi$,
  PyDTMC costruisce la **matrice dei flussi**

  $
    F_{ij}= \pi_i\,P_{ij}.
  $

  Se $F$ risulta (entro la tolleranza numerica) **simmetrica**
  $(F \approx F^{\mathsf{T}})$, la catena soddisfa il bilancio dettagliato
  e viene etichettata come **reversible = YES**.

In [ ]:
mc = MarkovChain(P)
print(mc)
print(mc.steady_states)

## Calcolo diretto di $\pi$ tramite sistemi lineari e autovalori

### Metodo dei sistemi lineari
- **DTMC**: risolve (I−Pᵀ)x=0  
- **CTMC**: risolve Qᵀx=0  


### Metodo eigen-solver
- π è l'autovettore **sinistro** associato a:
  - λ=1 per P (DTMC)  
  - λ=0 per Q (CTMC)

In [ ]:
pi1 = metodi_stazionaria.sistema_numpy(P)
pi2 = metodi_stazionaria.sistema_scipy(P)
pi3 = metodi_stazionaria.solve_via_eig_numpy(P)
pi4 = metodi_stazionaria.solve_via_eig_scipy(P)
print(pi1,pi2,pi3,pi4,sep="\n\n")

## Metodo Power, ad iterazioni

Algoritmo iterativo semplice ed efficace per trovare la distribuzione stazionaria.

### Passaggi

1. **Inizializzazione**  
   $[
   \pi^{(0)} = \left[ \tfrac{1}{n}, \dots, \tfrac{1}{n} \right]
   $]

2. **Iterazione**  
   $[
   \pi^{(t+1)} =
   \begin{cases}
   \pi^{(t)} P, & \text{(DTMC)} \\
   \pi^{(t)} \left(I + \frac{Q}{B}\right), & \text{(CTMC)}
   \end{cases}
   \quad
   \text{poi normalizzazione: } \pi^{(t+1)} \leftarrow \frac{\pi^{(t+1)}}{\sum_i \pi^{(t+1)}_i}
   $]

3. **Convergenza**  
   $[
   \| \pi^{(t+1)} - \pi^{(t)} \|_1 < \text{tol}
   \quad (\text{tol} = 10^{-15})
   $]

In [ ]:
pi1 = metodi_stazionaria.solve_via_power_numpy(P)
pi2 = metodi_stazionaria.solve_via_power_scipy(P)
print(pi1, pi2, sep="\n\n")

## Andamento precisione e costo computazionale

## Andamento precisione e costo computazionale

## Andamento precisione e costo computazionale

## Andamento precisione e costo computazionale

In [ ]:
# === Lista delle funzioni da confrontare ===
metodi = [     
    "gth",
    "system_scipy",
    "system_numpy",
    "eig_numpy",
    "eig_scipy",
    "power_numpy",
    "power_scipy"
]

# === Calcolo dati per ogni metodo ===
risultati = {}

for metodo in sorted(metodi):
    dati = complessità.calcola_costo_e_errore(
        funzione=metodo,
        C=100,
        n_punti=20,
        n_matrici=200,
        sparsity=0,
        k=5,
        tipo_gen_P_orQ=generazione_catene.genera_P_irriducibile,
        discrete=True
    )
    risultati[metodo] = {
        "x": [d[0] for d in dati],
        "tempo": [d[1] for d in dati],
        "errore": [d[2] for d in dati]
    }

In [ ]:
import matplotlib.ticker as ticker

# === Grafico 1: Tempo di esecuzione ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["tempo"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

# Mostra solo griglia principale
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')

# Forza i tick solo sulle decine
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Tempo medio (secondi)", fontsize=12)
ax.set_title("Confronto dei tempi di calcolo di $\pi$ su matrice P densa", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("P_DENSA.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

# === Grafico 2: Errore ||πP - π||₁ ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["errore"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Errore", fontsize=12)
ax.set_title("Confronto degli errori di calcolo di $\pi$ su matrice P densa", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("P_DENSA_ERRORE.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

In [ ]:
# === Lista delle funzioni da confrontare ===
metodi = [     
    "gth",
    "system_scipy",
    "system_numpy",
    "eig_numpy",
    "eig_scipy",
    "power_numpy",
    "power_scipy"
]

# === Calcolo dati per ogni metodo ===
risultati = {}

for metodo in sorted(metodi):
    dati = complessità.calcola_costo_e_errore(
        funzione=metodo,
        C=100,
        n_punti=20,
        n_matrici=200,
        sparsity=0,
        k=5,
        tipo_gen_P_orQ=generazione_catene.genera_P_k_random,
        discrete=True
    )
    risultati[metodo] = {
        "x": [d[0] for d in dati],
        "tempo": [d[1] for d in dati],
        "errore": [d[2] for d in dati]
    }

In [ ]:
import matplotlib.ticker as ticker

# === Grafico 1: Tempo di esecuzione ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["tempo"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

# Mostra solo griglia principale
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')

# Forza i tick solo sulle decine
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Tempo medio (secondi)", fontsize=12)
ax.set_title("Confronto dei tempi di calcolo di $\pi$ su matrice P sparsa con 5 archi per nodo", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("P_SPARSA.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

# === Grafico 2: Errore ||πP - π||₁ ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["errore"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Errore", fontsize=12)
ax.set_title("Confronto degli errori di calcolo di $\pi$ su matrice P sparsa con 5 archi per nodo", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("P_SPARSA_ERRORE.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

In [ ]:
# === Lista delle funzioni da confrontare ===
metodi = [     
    "gth",
    "system_scipy",
    "system_numpy",
    "eig_numpy",
    "eig_scipy",
    "power_numpy",
    "power_scipy"
]

# === Calcolo dati per ogni metodo ===
risultati = {}

for metodo in sorted(metodi):
    dati = complessità.calcola_costo_e_errore(
        funzione=metodo,
        C=100,
        n_punti=20,
        n_matrici=200,
        sparsity=0,
        k=5,
        tipo_gen_P_orQ=generazione_catene.genera_Q_irriducibile,
        discrete=False
    )
    risultati[metodo] = {
        "x": [d[0] for d in dati],
        "tempo": [d[1] for d in dati],
        "errore": [d[2] for d in dati]
    }

In [ ]:
import matplotlib.ticker as ticker

# === Grafico 1: Tempo di esecuzione ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["tempo"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

# Mostra solo griglia principale
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')

# Forza i tick solo sulle decine
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Tempo medio (secondi)", fontsize=12)
ax.set_title("Confronto dei tempi di calcolo di $\pi$ su matrice Q densa", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("Q_DESNA.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

# === Grafico 2: Errore ||πP - π||₁ ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["errore"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Errore", fontsize=12)
ax.set_title("Confronto degli errori di calcolo di $\pi$ su matrice Q densa", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("Q_DENSA_ERRORE.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

In [ ]:
# === Lista delle funzioni da confrontare ===
metodi = [     
    "gth",
    "system_scipy",
    "system_numpy",
    "eig_numpy",
    "eig_scipy",
    "power_numpy",
    "power_scipy"
]

# === Calcolo dati per ogni metodo ===
risultati = {}

for metodo in sorted(metodi):
    dati = complessità.calcola_costo_e_errore(
        funzione=metodo,
        C=100,
        n_punti=20,
        n_matrici=200,
        sparsity=0,
        k=5,
        tipo_gen_P_orQ=generazione_catene.genera_Q_k_random,
        discrete=False
    )
    risultati[metodo] = {
        "x": [d[0] for d in dati],
        "tempo": [d[1] for d in dati],
        "errore": [d[2] for d in dati]
    }

In [ ]:
import matplotlib.ticker as ticker

# === Grafico 1: Tempo di esecuzione ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["tempo"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

# Mostra solo griglia principale
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')

# Forza i tick solo sulle decine
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Tempo medio (secondi)", fontsize=12)
ax.set_title("Confronto dei tempi di calcolo di $\pi$ su matrice Q sparsa con 5 archi per nodo", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("Q_SPARSA.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()

# === Grafico 2: Errore ||πP - π||₁ ===
fig, ax = plt.subplots(figsize=(18,10))
for metodo in metodi:
    ax.plot(risultati[metodo]["x"], risultati[metodo]["errore"], 'o-', label=metodo, markersize=3)
ax.set_xscale('log')
ax.set_yscale('log')

ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(False, which='minor')
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))
ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, subs=(1.0,), numticks=100))

ax.set_xlabel("Dimensione matrice $n$", fontsize=12)
ax.set_ylabel("Errore", fontsize=12)
ax.set_title("Confronto degli errori di calcolo di $\pi$ su matrice Q sparsa con 5 archi per nodo", fontsize=14)
ax.legend(fontsize=13)


plt.savefig("Q_SPARSA_ERRORE.svg", dpi=300, format="svg", bbox_inches="tight")
plt.show()